# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

1. Load data
2. Assign points to clusters
3. compute cluster means
4. iterate, reassign, recompute kmeans

#### b) Create the builder function for step 1.

#### c) Create the builder function for step 2.

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

#### e) Create the builder function for step 4.

#### f) Combine them all into your own `kmeans` function.

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [13]:
voltages <- read.csv("voltages_df.csv")

#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [16]:
library(dplyr)
library(tibble)

label_randomly <- function(n_points, n_clusters){
  sample(((1:n_points) %% n_clusters)+1, n_points, replace=F)
}

get_cluster_means <- function(data, labels){
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(everything(), mean), .groups = "drop") %>%
    arrange(label__)
}

assign_cluster_fast <- function(data, means){
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% dplyr::select(-label__))
  dii <- sort(rep(1:nrow(data), nrow(means)))
  mii <- rep(1:nrow(means), nrow(data))
  data_repped <- data_matrix[dii, ]
  means_repped <- means_matrix[mii, ]
  diff_squared <- (data_repped - means_repped)^2
  all_distances <- rowSums(diff_squared)
  tibble(dii=dii, mii=mii, distance=all_distances) %>%
    group_by(dii) %>%
    arrange(distance) %>%
    filter(row_number()==1) %>%
    ungroup() %>%
    arrange(dii) %>%
    pull(mii)
}

assign_cluster_slow <- function(data, means){    
  dii <- 1:nrow(data)
  cii <- 1:nrow(means)
  labels <- c()
  for(point_index in dii){
    smallest_dist <- Inf
    smallest_label <- NA
    for(clus_index in cii){
      point <- data[point_index, ]
      clus <- means %>% dplyr::select(-label__) %>% `[`(clus_index, )
      diff <- point - clus
      dist <- sum(diff * diff)
      if(dist < smallest_dist){
        smallest_dist <- dist
        smallest_label <- means[clus_index, ]$label__
      }
    }
    labels <- c(labels, smallest_label)
  }        
  labels    
}

kmeans_done <- function(old_means, new_means, eps=1e-6){
  om <- as.matrix(old_means)
  nm <- as.matrix(new_means)
  m <- mean(sqrt(rowSums((om - nm)^2)))
  if(m < eps) TRUE else FALSE
}

mykmeans <- function(data, n_clusters, eps=1e-6, max_it = 1000, verbose = FALSE){
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE
  it <- 0
  while(!done & it < max_it){
    labels <- assign_cluster_fast(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if(kmeans_done(old_means, new_means)){
      done <- TRUE
    } else {
      old_means <- new_means
      it <- it + 1
      if(verbose){
        cat(sprintf("%d\n", it))
      }
    }
  }
  list(labels=labels, means=new_means)
}


results <- mykmeans(voltages, n_clusters = 3)
print(results$labels)
print(results$means)

  [1] 2 3 3 3 3 3 1 1 3 2 2 2 1 2 3 2 2 2 2 2 3 3 3 3 3 2 2 2 1 1 1 1 3 1 1 3 3
 [38] 2 1 3 2 2 2 1 3 1 1 3 1 1 3 1 2 2 1 2 3 2 1 3 2 2 3 2 1 1 3 1 2 1 2 3 1 1
 [75] 3 3 2 1 1 1 1 1 3 1 1 3 3 3 2 1 1 1 3 1 3 3 1 3 3 1 1 1 3 1 2 2 1 2 2 3 2
[112] 2 3 1 2 1 2 2 3 3 3 3 2 2 1 3 3 1 2 1 3 2 3 1 2 3 3 3 2 1 2 3 3 1 2 2 3 2
[149] 2 1 2 1 3 3 1 2 1 1 3 2 1 1 2 3 2 1 3 2 3 2 2 1 3 2 1 1 3 1 2 3 1 3 1 2 1
[186] 3 3 2 3 2 2 3 2 2 1 1 2 3 2 2 1 2 1 1 3 2 3 1 3 2 3 1 2 1 1 3 1 1 3 3 3 3
[223] 3 3 3 1 1 1 1 2 3 2 1 3 3 1 1 2 2 1 1 2 1 1 3 1 2 1 1 3 1 3 2 1 3 1 1 2 2
[260] 2 3 3 1 2 3 3 3 1 1 1 2 3 3 3 1 2 1 1 3 3 1 2 1 2 2 3 1 3 3 1 1 2 1 1 2 3
[297] 2 1 1 3 3 1 2 2 2 1 2 3 3 3 2 2 2 2 2 2 3 2 3 2 2 1 3 1 2 2 2 2 3 2 3 2 1
[334] 2 3 3 1 1 2 3 1 3 1 1 2 1 3 1 2 2 1 3 3 3 2 1 3 2 1 1 3 3 3 3 2 1 2 3 2 2
[371] 2 2 2 3 3 1 1 2 2 2 2 1 1 2 2 1 1 3 3 1 2 3 3 3 3 1 3 3 1 2 2 2 2 3 3 2 2
[408] 1 1 1 3 1 1 2 3 3 3 1 3 3 3 2 2 3 2 1 3 3 3 3 1 1 3 1 2 3 2 2 2 1 1 1 1 2
[445] 3 2 2 2 1 3 2 2 2 3 3 2 3 3 2 1 3 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [4]:
voltage_matrix <- as.matrix(voltages)

results <- kmeans(voltage_matrix, centers = 3)

print(results$cluster)
print(results$centers)

  [1] 3 2 2 2 2 1 3 3 2 3 3 3 3 3 2 3 3 3 3 3 1 2 2 2 2 3 3 3 3 3 3 3 2 3 3 2 2
 [38] 3 3 1 3 3 3 3 1 3 3 2 3 3 1 3 3 3 3 3 1 3 3 2 3 3 2 3 3 3 1 3 3 3 3 2 3 3
 [75] 2 1 3 3 3 3 3 3 2 3 3 2 2 2 3 3 3 3 2 3 2 1 3 2 2 3 3 3 2 3 3 3 3 3 3 1 3
[112] 3 1 3 3 3 3 3 2 2 1 2 3 3 3 1 2 3 3 3 2 3 2 3 3 1 2 1 3 3 3 2 2 3 3 3 2 3
[149] 3 3 3 3 2 2 3 3 3 3 1 3 3 3 3 2 3 3 2 3 2 3 3 3 2 3 3 3 2 3 3 1 3 2 3 3 3
[186] 1 1 3 1 3 3 1 3 3 3 3 3 1 3 3 3 3 3 3 2 3 2 3 1 3 1 3 3 3 3 2 3 3 2 2 2 1
[223] 2 2 1 3 3 3 3 3 2 3 3 2 2 3 3 3 3 3 3 3 3 3 2 3 3 3 3 1 3 2 3 3 1 3 3 3 3
[260] 3 2 2 3 3 2 2 2 3 3 3 3 1 1 1 3 3 3 3 1 2 3 3 3 3 3 1 3 2 2 3 3 3 3 3 3 1
[297] 3 3 3 2 2 3 3 3 3 3 3 1 2 1 3 3 3 3 3 3 2 3 2 3 3 3 2 3 3 3 3 3 2 3 1 3 3
[334] 3 1 2 3 3 3 2 3 2 3 3 3 3 2 3 3 3 3 2 1 2 3 3 2 3 3 3 1 2 2 2 3 3 3 1 3 3
[371] 3 3 3 2 1 3 3 3 3 3 3 3 3 3 3 3 3 1 2 3 3 2 2 1 2 3 1 2 3 3 3 3 3 2 2 3 3
[408] 3 3 3 2 3 3 3 2 1 1 3 2 2 2 3 3 1 3 3 2 1 2 1 3 3 1 3 3 1 3 3 3 3 3 3 3 3
[445] 2 3 3 3 3 2 3 3 3 1 2 3 2 2 3 3 2 

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

- The labels and clusters are slightly different because of the manual computation that is being completed with the loops to determine centroids. The means are also slightly different. 

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

- For each data point, we loop over all clusters and then compute the distance between the point and each cluster mean such that there is a double loop of N over K.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

- Two matrices are created to compare all cluster-points at the same time where the matrices are then subtracted to compute the distances.

#### c) State which (for loops or vectorizing) is more efficient and why.

- Vectorizing is more efficient because index vectors are reptitively created without the use of a loop, which allows more efficiency in developing replicated matrices.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

- kmeans fails when clusters are not the same size or if there is no predefined K number of centers.
- The assumptions that would cause failure are that the clusters are all spherical and of the same size and a predefined K is set.

## Question 5
#### What assumption do Guassian mixture models make?

- The assumption is that data is drawn from N Gaussian distributions with individual parameters estimated from the data.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

- The assumption is that two points are likely to be in the same cluster if close to one another.
- This helps us because it is a singular assumption made and is only a metric of the original data that is used.

## Question 7
#### Define the gap statistic method. What do we use it for?

- This is comparing the clustering of each value to a cluster of randomized data in the same domain as the original data.
- We use this for standard comparisons to look at the most basic, best fit model.